<a href="https://colab.research.google.com/github/karanja2/Random_forest_classification/blob/main/churn_moddeling_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **importing the libraries**


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# **loading the Data set**

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Churn_Modelling.csv')


In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data.shape


(10000, 14)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


# **checking the Data set**

In [6]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

just droping unnecessary column

In [42]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# **Feature Engineering**

 One Hot Encoding Transforming Chategorical Data to Numeric

In [43]:
data.Gender.unique()



array(['Female', 'Male'], dtype=object)

In [44]:
pd.get_dummies(data['Gender']).head()

,Female,Male
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [45]:
Gender = pd.get_dummies(data['Gender'], drop_first=True)
Gender.head()

,Male
0,0
1,0
2,0
3,0
4,0


i have droped the first column because only one column is sufficient to determine the gender of the customers either male(1) or female(0)

In [46]:
data.Geography.unique()



array(['France', 'Spain', 'Germany'], dtype=object)

In [48]:
pd.get_dummies(data['Geography']).head()


,France,Germany,Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1


In [49]:
Geography = pd.get_dummies(data['Geography'],drop_first = True)
Geography.head(5)

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1


# concatenate the above two data to the original df

In [50]:
new_data=pd.concat([data,Gender,Geography], axis=1)

In [51]:
new_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,0,1


# drop the repeated columns

In [52]:
new_data.drop(['Gender','Geography'],axis=1, inplace=True)

In [53]:
new_data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


# creating separate target variable

In [54]:
Y=new_data['Exited']
Y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [55]:
Y.unique()

array([1, 0])

# Droping the target variable from the dataset

In [56]:

X=new_data.drop(['Exited'],axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1


# splitting the dataset into the Training set and Test set

In [57]:
from sklearn.model_selection import train_test_split


In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [59]:
print("X_train", X_train.shape)
print("X_test", X_test.shape)
print("Y_train", Y_train.shape)
print("Y_test", Y_test.shape)

X_train (8000, 11)
X_test (2000, 11)
Y_train (8000,)
Y_test (2000,)


# Training Random Forest Classification On Training set

In [60]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=5, criterion="entropy", random_state=0)
classifier.fit(X_train,Y_train)


RandomForestClassifier(criterion='entropy', n_estimators=5, random_state=0)

# Predicting the Test Result

In [61]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 1, 0])

# confusion matrix

In [62]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,accuracy_score

In [64]:
cm=confusion_matrix(Y_test,y_pred)
print(cm)

[[1484  111]
 [ 192  213]]


In [65]:
accuracy_score(Y_test,y_pred)

0.8485

In [68]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      1595
           1       0.67      0.51      0.58       405

    accuracy                           0.85      2000
   macro avg       0.78      0.72      0.74      2000
weighted avg       0.84      0.85      0.84      2000



# Try with different Numbers of Trees and gini

In [69]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = "gini",random_state = 0)
classifier.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=10, random_state=0)

In [70]:
y_pred = classifier.predict(X_test)

In [71]:
accuracy_score (Y_test, y_pred)

0.855

# with more numbers of trees and gini criteria, the accuracy increases